# Reactor map (GeoNuclearData)

Interactive map and tables of global reactor units from GeoNuclearData.


In [3]:
import pandas as pd
from pathlib import Path
import plotly.express as px

# Resolve data root whether running from repo root or notebooks dir
cwd = Path.cwd().resolve()
if (cwd / "external" / "GeoNuclearData").exists():
    DATA_ROOT = cwd / "external" / "GeoNuclearData" / "data" / "csv" / "raw"
elif cwd.name == "notebooks" and (cwd.parent / "external" / "GeoNuclearData").exists():
    DATA_ROOT = cwd.parent / "external" / "GeoNuclearData" / "data" / "csv" / "raw"
else:
    raise RuntimeError("Cannot locate external/GeoNuclearData; run from repo root or notebooks directory.")

countries = pd.read_csv(DATA_ROOT / "1-countries.csv")
status_types = pd.read_csv(DATA_ROOT / "2-nuclear_power_plant_status_type.csv")
reactor_types = pd.read_csv(DATA_ROOT / "3-nuclear_reactor_type.csv")
plants = pd.read_csv(DATA_ROOT / "4-nuclear_power_plants.csv")

status_map = dict(zip(status_types.Id, status_types.Type))
type_map = dict(zip(reactor_types.Id, reactor_types.Type))
country_map = dict(zip(countries.Code, countries.Name))

df = plants.copy()
df["status"] = df["StatusId"].map(status_map)
df["reactor_type"] = df["ReactorTypeId"].map(type_map)
df["country"] = df["CountryCode"].map(country_map)
df["capacity_mw"] = df["Capacity"]
date_cols = ["ConstructionStartAt", "OperationalFrom", "OperationalTo", "LastUpdatedAt"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df.head()

C:\Users\jws\AppData\Local\Temp\ipykernel_9124\498998647.py:31: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[col] = pd.to_datetime(df[col], errors="coerce")


,Id,Name,Latitude,Longitude,CountryCode,StatusId,ReactorTypeId,ReactorModel,ConstructionStartAt,OperationalFrom,OperationalTo,Capacity,Source,LastUpdatedAt,IAEAId,status,reactor_type,country,capacity_mw
0,1,Ågesta,59.206000,18.08290,SE,5,20.0,NaN,1957-12-01,1964-05-01,1974-06-02,9.0,WNA/IAEA,2015-05-24 04:51:37+03:00,528.0,Shutdown,PHWR,Sweden,9.0
1,2,Akademik Lomonosov-1,69.709579,170.30625,RU,3,21.0,KLT-40S 'Floating',2007-04-15,2020-05-22,NaT,30.0,WNA/IAEA/Google Maps,NaT,895.0,Operational,PWR,Russia,30.0
2,3,Akademik Lomonosov-2,69.709579,170.30625,RU,3,21.0,KLT-40S 'Floating',2007-04-15,2020-05-22,NaT,30.0,WNA/IAEA/Google Maps,NaT,896.0,Operational,PWR,Russia,30.0
3,4,Akhvaz-1,NaN,NaN,IR,1,NaN,NaN,NaT,NaT,NaT,NaN,WNA,NaT,NaN,Planned,NaN,Iran,NaN
4,5,Akhvaz-2,NaN,NaN,IR,1,NaN,NaN,NaT,NaT,NaT,NaN,WNA,NaT,NaN,Planned,NaN,Iran,NaN


In [4]:
print("Units:", len(df))
print("Countries:", df["CountryCode"].nunique())
print("Status counts:", df["status"].value_counts().sort_values(ascending=False))
print("Reactor type counts:", df["reactor_type"].value_counts().head(10))

Units: 804
Countries: 41
Status counts: status
Operational                  414
Shutdown                     209
Planned                       79
Under Construction            61
Suspended Operation           25
Suspended Construction         6
Cancelled Construction         4
Decommissioning Completed      3
Never Commissioned             2
Unknown                        1
Name: count, dtype: int64
Reactor type counts: reactor_type
PWR      484
BWR      120
PHWR      74
GCR       52
LWGR      25
FBR       19
ABWR       6
HTGR       5
HWGCR      4
HWLWR      2
Name: count, dtype: int64


## Map – unit-level
Sized by capacity (MW), colored by status. Hover shows type, model, dates.


In [ ]:
map_df = df.dropna(subset=["Latitude", "Longitude", "capacity_mw"]).copy()
map_df["hover"] = (
    map_df["Name"]
    + " | "
    + map_df["status"].fillna("?")
    + " | "
    + map_df["reactor_type"].fillna("?")
    + " | model="
    + map_df["ReactorModel"].fillna("")
)

fig = px.scatter_geo(
    map_df,
    lon="Longitude",
    lat="Latitude",
    size="capacity_mw",
    color="status",
    hover_name="Name",
    hover_data={
        "capacity_mw": True,
        "country": True,
        "reactor_type": True,
        "ReactorModel": True,
        "OperationalFrom": True,
        "OperationalTo": True,
    },
    size_max=18,
    title="Reactor units (GeoNuclearData)",
)
fig.update_layout(legend_title_text="Status")
fig.show()


## Map – aggregated by site
Units combined per (name, country, lat/lon); status = most common among its units.


In [ ]:
def mode_or_first(series: pd.Series):
    series = series.dropna()
    if series.empty:
        return None
    return series.mode().iat[0]

agg = (
    df.dropna(subset=["Latitude", "Longitude", "capacity_mw"])
    .groupby(["Name", "CountryCode", "Latitude", "Longitude"], as_index=False)
    .agg(
        capacity_mw=("capacity_mw", "sum"),
        units=("Id", "count"),
        status=("status", mode_or_first),
        reactor_types=("reactor_type", lambda s: ", ".join(sorted(set(s.dropna())))),
    )
)
agg["country"] = agg["CountryCode"].map(country_map)

fig = px.scatter_geo(
    agg,
    lon="Longitude",
    lat="Latitude",
    size="capacity_mw",
    color="status",
    hover_name="Name",
    hover_data={
        "capacity_mw": True,
        "units": True,
        "country": True,
        "reactor_types": True,
    },
    size_max=20,
    title="Reactor sites (units aggregated)",
)
fig.update_layout(legend_title_text="Status")
fig.show()


## Map – Plotly Mapbox (tile background)
Uses OpenStreetMap tiles (no token needed). Set `status_filter` to focus on selected statuses.


In [ ]:
status_filter = []  # e.g., ["Operational", "Under construction"]
if status_filter:
    mapbox_df = map_df[map_df["status"].isin(status_filter)].copy()
else:
    mapbox_df = map_df.copy()

if mapbox_df.empty:
    print("No units after status filter.")
else:
    fig = px.scatter_mapbox(
        mapbox_df,
        lat="Latitude",
        lon="Longitude",
        size="capacity_mw",
        color="status",
        hover_name="Name",
        hover_data={
            "capacity_mw": True,
            "country": True,
            "reactor_type": True,
            "ReactorModel": True,
            "OperationalFrom": True,
            "OperationalTo": True,
        },
        size_max=16,
        zoom=1.2,
        title="Reactor units – Mapbox",
    )
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()


## Largest sites by capacity


In [ ]:
agg.sort_values("capacity_mw", ascending=False).head(20)


## Capacity by country (top 20)


In [ ]:
country_cap = df.groupby("CountryCode", as_index=False).agg(capacity_mw=("capacity_mw", "sum"), units=("Id", "count"))
country_cap["country"] = country_cap["CountryCode"].map(country_map)
country_cap.sort_values("capacity_mw", ascending=False).head(20)
